# Stocks-ETL-System Notebook PlayGround
This notebook will demonstrate the necessary functions needed to be able to download PH-Stocks Data from MarketWatch.com & Inquirer.net which would be updated maintained using the necessary data-focused python pipelines (which will also have their specific functionsbe included to this file as well).

The architecture of such data storage is based on the principle of ETL/ELT itself, which (depending which type) consists of 3 layered storage for maximum usage of data:
1) Data Lakes

2) Data Warehouse

3) Data Marts 

### Importing section
To ensure maxmimum usage of python's capabilities within this playground, I personally have imported all of the libraries that I may be able to use.

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time
import datetime
import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Variable Initialization
This section is for informing the system of the needed variables it needs to play with; that includes links for scraping, and a function that simplifies the driver usage 

In [15]:
mw_list = 'https://www.marketwatch.com/tools/markets/stocks/country/philippines'
mw_main_link = 'https://www.marketwatch.com'
pb_link = 'https://www.pesobility.com/stock/{s_type}'

In [69]:
inq_mains='https://www.inquirer.net/search/?q={company_n}'

In [70]:
import dataHandler as handler

app = handler.dataHandler()
app.inq_setNewsData(company_name='Aboitiz Equity Ventures',inq_main=inq_mains)

([{'news_link': 'https://business.inquirer.net/292254/aev-nets-p22b',
   'news_date_published': ('3/11/2020',
    <dataHandler.dataHandler at 0x2af95eba460>),
   'news_title': 'Aboitiz Equity nets P22B | Inquirer Business',
   'comp_name': 'Aboitiz Equity Ventures'},
  {'news_link': 'https://business.inquirer.net/275827/aboitiz-equity-ventures-nets-p9b-in-h1',
   'news_date_published': ('7/31/2019',
    <dataHandler.dataHandler at 0x2af95eba460>),
   'news_title': 'Aboitiz Equity Ventures nets P9B in H1 | Inquirer Business',
   'comp_name': 'Aboitiz Equity Ventures'},
  {'news_link': 'https://business.inquirer.net/319596/aboitiz-group-sets-p48-b-capital-spending-for-2021',
   'news_date_published': ('3/17/2021',
    <dataHandler.dataHandler at 0x2af95eba460>),
   'news_title': 'Aboitiz group sets P48-B capital spending for 2021 | Inquirer Business',
   'comp_name': 'Aboitiz Equity Ventures'},
  {'news_link': 'https://business.inquirer.net/332493/jollibee-ayala-aboitiz-among-worlds-best

In [11]:
def setSelenium():
    options=Options()
    options.add_argument('start-maximized')
    options.add_argument('--headless')
    driver=webdriver.Firefox(options=options,executable_path=r'Y:\geckodriver.exe')
    return driver

In [92]:
def stringdate_toDate(date=None):
    if date==None:
        date = str(datetime.datetime.now()).split()[0].split('-')
        year = date[0]
        month = date[1]
        day = date[2]
        return '{mm}/{dd}/{yyyy}'.format(mm=month,dd=day,yyyy=year)
    month_list = ['January','February','March','April','May','June','July','August','September','October','November','December']
    month_dict = dict((y,x+1) for x,y in enumerate(month_list))
    month = date.replace(',','').split()[0]
    day = date.replace(',','').split()[1]
    year = date.replace(',','').split()[2]
    for i in month_list:
        if i.startswith(str(month)):
            month = month_dict[i]
    return '{mm}/{dd}/{yyyy}'.format(mm=month,dd=day,yyyy=year)
stringdate_toDate('Mar 12, 2021')

'3/12/2021'

### Inquirer.net Handlers
This section is for creating necessary functions to webscrape Inquirer.net, and that task includes these several sub-tasks (one for the datalake and atawarehouse):
##### Data Lake/Data Warehouse
- scraping news_links & metadata (#inq_setNewsData) [basically used for datalake, which can also be used for first initialization]

##### 1st Pipeline (dl-dw)
- scraping news_content from the taken fromo each news_link (#extract_newsContent) [used for dl-dw pipeline workload]

##### 2nd Pipeline
- nlp solution

In [18]:
def inq_setNewsData(company_name,inq_main,driver=None):
    inq_link = inq_main.format(company_n=company_name.lower())
    cycle_pages = []
    myDriver = driver if driver != None else setSelenium()
    myDriver.get(inq_link)
    page = BeautifulSoup(myDriver.page_source,'html.parser')
    mainpage = myDriver.current_url
    myDriver.quit()
    cycle_pages.append(page)
    navi=page.find('div',attrs={'class':'gsc-cursor'})
    time.sleep(2)
    for i,pages in enumerate(navi.find_all('div',attrs={'class':'gsc-cursor-page'})):
        time.sleep(6)
        myDriver2 = driver if driver != None else setSelenium()
        myDriver2.get(mainpage[:-1]+str(pages.text))
        page2 = BeautifulSoup(myDriver2.page_source,'html.parser')
        cycle_pages.append(page2)
        myDriver2.quit()
    

    data = []
    for i in cycle_pages:
        box=i.find('div',attrs={'class':'gsc-expansionArea'})
        mini_boxes = box.find_all('div',attrs={'class':'gsc-webResult gsc-result'})
        for i in mini_boxes:
            links = i.find('a')['href']
            title = i.find('a').text
            date = i.find('div',attrs={'class':'gs-bidi-start-align gs-snippet'}).text
            date_f = stringdate_toDate(date[:date.index('.')].strip())
            comp_name = company_name
            data.append({'news_link':links,'news_date_published':date_f,'news_title':title,'comp_name':comp_name})
    
    return data
    
inq_setNewsData('Aboitiz Equity Ventures',inq_main)

[{'news_link': 'https://business.inquirer.net/292254/aev-nets-p22b',
  'news_date_published': '3/11/2020',
  'news_title': 'Aboitiz Equity nets P22B | Inquirer Business'},
 {'news_link': 'https://business.inquirer.net/275827/aboitiz-equity-ventures-nets-p9b-in-h1',
  'news_date_published': '7/31/2019',
  'news_title': 'Aboitiz Equity Ventures nets P9B in H1 | Inquirer Business'},
 {'news_link': 'https://business.inquirer.net/319596/aboitiz-group-sets-p48-b-capital-spending-for-2021',
  'news_date_published': '3/17/2021',
  'news_title': 'Aboitiz group sets P48-B capital spending for 2021 | Inquirer Business'},
 {'news_link': 'https://business.inquirer.net/332493/jollibee-ayala-aboitiz-among-worlds-best-employers',
  'news_date_published': '10/15/2021',
  'news_title': "Jollibee, Ayala, Aboitiz among world's 'best employers' | Inquirer ..."},
 {'news_link': 'https://business.inquirer.net/269641/aboitiz-equity-ventures-inc-nets-p3-5b-in-q1-2019',
  'news_date_published': '5/2/2019',
  'n

In [33]:
def extract_newsContent(inq_link,driver=None):
    #add with the parameters comp_name
    #add a functionality to find comp
    content=''
    myDriver = driver if driver != None else setSelenium()
    myDriver.get(inq_link)
    page = BeautifulSoup(myDriver.page_source,'html.parser')
    myDriver.quit()
    container = page.find('div',attrs={'id':'article_content'})
    for i in container.find_all('p'):
        if i.text == "":
            continue
        content+=i.text
    return content

extract_newsContent('https://business.inquirer.net/292254/aev-nets-p22b')

'Conglomerate Aboitiz Equity Ventures (AEV) ended last year with a 1-percent decline in net income amounting to P22 billion due to slower earnings from its flagship power business.Power accounted for 57 percent of the total income contributions, followed by banking and financial services with a 30 percent share. Food, property and infrastructure businesses accounted for 7 percent, 4 percent and 2 percent, respectively.“Despite challenges in our power business last year, our portfolio was largely able to sustain its operational and bottom line performance, as better-performing businesses continued their positive trajectory,” AEV president and chief executive officer Sabin Aboitiz said.l“As we commemorate the 100th year of our majority owner, Aboitiz & Co., we will continue to build a well-diversified and resilient investment portfolio into the next decade by advancing business and communities,” Aboitiz added.Aboitiz Power Corp.’s net income contribution to AEV for 2019 decreased by 20 p

### MarketWatch.com Handlers
This section, similar to the one before this, focuses on handling the needed functionalities for web-scraping and data handling needed for the website MarketWatch.com, this would also include functions needed for pipeline workload handling:

##### Data Lake
- scraping company_overview (#extract_overview)


##### Data Warehouse
- scraping stocks_metadata (#mw_setStockData)
- scraping company_financial_profile (#extract_finProf)
- scraping historical_data (#extract_histQuotes)

##### 1st Pipeline (dl-dw)
- - 

##### 2nd Pipeline (dw-dm)
- - 



In [18]:
def mw_setStockData(mw_link,mw_main_link,pages=2,driver=None):
    page_data = []
    metadata = []
    
    for i in range(1,pages+1):
        cur_link = mw_link+'/{num}'.format(num=i)
        myDriver = driver if driver != None else setSelenium()
        myDriver.get(mw_link)
        mw_page = BeautifulSoup(myDriver.page_source,'html.parser')
        page_data.append(mw_page)
        myDriver.quit()
    for mw_page in page_data:    
        for i in mw_page.find_all('tr'):
            scrape_ins = i.find_all('td')
            if scrape_ins ==[]:
                continue
            data_ins = {
                'name':scrape_ins[0].find('a').text,
                'symbol':scrape_ins[0].find('small').text[1:-1],
                'link':main_link+scrape_ins[0].find('a')['href'],
                'f_exchange':scrape_ins[1].text,
                'sector':scrape_ins[2].text
            }
            metadata.append(data_ins)
    
    return metadata
mw_setStockData(mw_list,mw_main_link)

[{'name': '2Go Group Inc. (2GO)',
  'symbol': '2GO',
  'link': 'https://www.marketwatch.com/investing/Stock/2GO?countryCode=PH',
  'f_exchange': 'XPHS',
  'sector': 'Water Transport/Shipping'},
 {'name': '8990 Holdings Inc. (HOUSE)',
  'symbol': 'HOUSE',
  'link': 'https://www.marketwatch.com/investing/Stock/HOUSE?countryCode=PH',
  'f_exchange': 'XPHS',
  'sector': 'Residential Building Construction'},
 {'name': '8990 Holdings Inc. Pfd. (8990P)',
  'symbol': '8990P',
  'link': 'https://www.marketwatch.com/investing/Stock/8990P?countryCode=PH',
  'f_exchange': 'XPHS',
  'sector': 'Residential Building Construction'},
 {'name': '8990 Holdings Inc. Pfd. B (8990B)',
  'symbol': '8990B',
  'link': 'https://www.marketwatch.com/investing/Stock/8990B?countryCode=PH',
  'f_exchange': 'XPHS',
  'sector': 'Residential Building Construction'},
 {'name': 'A. Brown Co. Inc. (BRN)',
  'symbol': 'BRN',
  'link': 'https://www.marketwatch.com/investing/Stock/BRN?countryCode=PH',
  'f_exchange': 'XPHS',

In [93]:
def extract_overview(stock_link,driver=None):
    overview_data=[]
    myDriver = driver if driver != None else setSelenium()
    myDriver.get(stock_link)
    page = BeautifulSoup(myDriver.page_source,'html.parser')
    myDriver.quit()
    over_list = page.find('ul',attrs={'class':'list list--kv list--col50'})
    cur_price1 = page.find('h2',attrs={'class':'intraday__price'})
    cur_status1 = page.find('div',attrs={'class':'element element--intraday'})
    cur_volume1 = page.find('div',attrs={'class':'range__header'})
    
    cur_status = cur_status1.find('div',attrs={'class':'status'}).text
    cur_price = cur_price1.find('span',attrs={'class':'value'}).text
    cur_volume = cur_volume1.find('span',attrs={'class':'primary'}).text
    cur_lastPrice = cur_status1.find('td',attrs={'class':'table__cell u-semi'}).text
    
    data_current = {
        'overview_date':stringdate_toDate(),
        'overview_statusNow':cur_status,
        'overview_priceNow':cur_price,
        'overview_volumeNow':cur_volume,
        'overview_lastpriceNow':cur_lastPrice
    }
    for i in over_list.find_all('li'):
        try:
            title = i.find('small',attrs={'class':'label'}).text
            data = i.find('span',attrs={'class':'primary'}).text
        except AttributeError:
            continue
        data_current['overview_'+title.replace(' ','_').lower().replace('/', '').replace('%','percent').replace('.','').replace('-','_')] = data
    return data_current
extract_overview('https://www.marketwatch.com/investing/Stock/2GO?countryCode=PH')

{'overview_date': '11/26/2021',
 'overview_statusNow': 'Closed',
 'overview_priceNow': '7.99',
 'overview_volumeNow': 'Volume: 9.3K',
 'overview_lastpriceNow': '₱7.80',
 'overview_open': '₱7.79',
 'overview_day_range': '7.79 - 7.99',
 'overview_52_week_range': '7.77 - 10.86',
 'overview_market_cap': '₱19.2B',
 'overview_shares_outstanding': 'N/A',
 'overview_public_float': '292.73M',
 'overview_beta': '0.55',
 'overview_rev_per_employee': 'N/A',
 'overview_pe_ratio': 'N/A',
 'overview_eps': '-₱0.70',
 'overview_yield': 'N/A',
 'overview_dividend': 'N/A',
 'overview_ex_dividend_date': 'N/A',
 'overview_short_interest': 'N/A',
 'overview_percent_of_float_shorted': 'N/A',
 'overview_average_volume': '29.1K'}

In [8]:
def extract_finProf(stock_link,driver=None):
    myDriver = driver if driver != None else setSelenium()
    myDriver.get(stock_link)
    page = BeautifulSoup(myDriver.page_source,'html.parser')
    myDriver.quit()
    title_holder = ''
    finProf_data = []
    fin_data ={}
    for i in page.find_all('div',attrs={'class':'element element--table'}):
        header=i.find('header',attrs={'class':'header header--secondary'})
        head = header.find('span',attrs={'class':'label'}).text.lower()[:3]
        table = i.find('table')
        for i,x in enumerate(table.find_all('td')):
            if (i+1)%2==0:
                fin_data[head+'_'+title_holder.replace(' ','_').lower().replace('/','').replace('-','').replace('(','').replace(')','')] = x.text
            title_holder=x.text
    return fin_data
extract_finProf('https://www.marketwatch.com/investing/stock/2go/company-profile?countrycode=ph&mod=mw_quote_tab')

{'val_pe_current': 'N/A',
 'val_pe_ratio_w_extraordinary_items': 'N/A',
 'val_pe_ratio_wo_extraordinary_items': 'N/A',
 'val_price_to_sales_ratio': '1.17',
 'val_price_to_book_ratio': '12.72',
 'val_price_to_cash_flow_ratio': '67.93',
 'val_enterprise_value_to_ebitda': '26.88',
 'val_enterprise_value_to_sales': '1.55',
 'val_total_debt_to_enterprise_value': '0.25',
 'eff_revenueemployee': 'N/A',
 'eff_income_per_employeee': 'N/A',
 'eff_receivables_turnover': '2.90',
 'eff_total_asset_turnover': '1.12',
 'liq_current_ratio': '0.93',
 'liq_quick_ratio': '0.85',
 'liq_cash_ratio': '0.10',
 'pro_gross_margin': '2.18%',
 'pro_operating_margin': '-5.34%',
 'pro_pretax_margin': '-10.22%',
 'pro_net_margin': '-10.58%',
 'pro_return_on_assets': '-11.83%',
 'pro_return_on_equity': '-72.24%',
 'pro_return_on_total_capital': '-10.00%',
 'pro_return_on_invested_capital': '-28.74%',
 'cap_total_debt_to_total_equity': '412.78',
 'cap_total_debt_to_total_capital': '80.50',
 'cap_total_debt_to_total_a

In [ ]:
def loop_hist2020()

In [13]:
def extract_histQuotes(stock_link,driver=None):
    myDriver = driver if driver != None else setSelenium()
    myDriver.get(stock_link)
    page = BeautifulSoup(myDriver.page_source,'html.parser')
    myDriver.quit()
    histQuotes = []
    table = page.find('table',attrs={'class':'table table--overflow align--center'})
    for i in table.find_all('tr')[1:]:
        try:
            date = i.find_all('td')[0].find('div').text
            open_p = i.find_all('td')[1].find('div').text
            high_p = i.find_all('td')[2].find('div').text
            low_p = i.find_all('td')[3].find('div').text
            close_p = i.find_all('td')[4].find('div').text
            volume = i.find_all('td')[5].find('div').text
        except AttributeError:
            continue
        data_full = {
            'date':date,
            'open_price':open_p,
            'high_price':high_p,
            'low_price':low_p,
            'closed_price':close_p,
            'volume':volume
        }
        histQuotes.append(data_full)
    return histQuotes
extract_histQuotes('https://www.marketwatch.com/investing/stock/2go/download-data?startDate=9/27/2021&endDate=10/1/2021&countryCode=ph')

[{'date': '11/25/2021',
  'open_price': '₱7.80',
  'high_price': '₱7.80',
  'low_price': '₱7.80',
  'closed_price': '₱7.80',
  'volume': '2,400'},
 {'date': '11/24/2021',
  'open_price': '₱7.85',
  'high_price': '₱7.85',
  'low_price': '₱7.80',
  'closed_price': '₱7.80',
  'volume': '15,300'},
 {'date': '11/23/2021',
  'open_price': '₱8.00',
  'high_price': '₱8.00',
  'low_price': '₱7.79',
  'closed_price': '₱7.85',
  'volume': '8,200'},
 {'date': '11/22/2021',
  'open_price': '₱8.00',
  'high_price': '₱8.00',
  'low_price': '₱7.99',
  'closed_price': '₱7.99',
  'volume': '4,600'},
 {'date': '11/19/2021',
  'open_price': '₱7.80',
  'high_price': '₱8.00',
  'low_price': '₱7.78',
  'closed_price': '₱7.78',
  'volume': '24,400'},
 {'date': '11/18/2021',
  'open_price': '₱7.92',
  'high_price': '₱7.92',
  'low_price': '₱7.80',
  'closed_price': '₱7.80',
  'volume': '19,100'},
 {'date': '11/17/2021',
  'open_price': '₱8.00',
  'high_price': '₱8.00',
  'low_price': '₱7.96',
  'closed_price':

In [3]:
import dataHandler as handler

In [14]:
mw_list = 'https://www.marketwatch.com/tools/markets/stocks/country/philippines'
mw_mainlink = 'https://www.marketwatch.com'
inq_mains='https://www.inquirer.net/search/?q={company_n}'
news_content = 'https://business.inquirer.net/292254/aev-nets-p22b'
comp_name = 'Aboitiz Equity Ventures'
overview_comp = 'https://www.marketwatch.com/investing/Stock/2GO?countryCode=PH'
finProf_comp = 'https://www.marketwatch.com/investing/stock/2go/company-profile?countrycode=ph&mod=mw_quote_tab'
histProf_comp = 'https://www.marketwatch.com/investing/stock/2go/download-data?countrycode=ph&mod=mw_quote_tab'

In [16]:
hd = handler.dataHandler()
#hd.stringdate_toDate()
#hd.inq_setNewsData(company_name=comp_name,inq_main=inq_mains)
#hd.extract_newsContent(comp_name=comp_name,inq_link=news_content)
#hd.mw_setStockData(mw_link=mw_list,main_link=mw_mainlink)
#hd.extract_overview(stock_link=overview_comp)
#hd.extract_finProf(stock_link=finProf_comp)
df = hd.extract_histQuotes(stock_link=histProf_comp)

In [17]:
pd.DataFrame.from_dict(df)

,date,open_price,high_price,low_price,closed_price,volume
0,11/25/2021,₱7.80,₱7.80,₱7.80,₱7.80,"2,400"
1,11/24/2021,₱7.85,₱7.85,₱7.80,₱7.80,"15,300"
2,11/23/2021,₱8.00,₱8.00,₱7.79,₱7.85,"8,200"
3,11/22/2021,₱8.00,₱8.00,₱7.99,₱7.99,"4,600"
4,11/19/2021,₱7.80,₱8.00,₱7.78,₱7.78,"24,400"
5,11/18/2021,₱7.92,₱7.92,₱7.80,₱7.80,"19,100"
6,11/17/2021,₱8.00,₱8.00,₱7.96,₱7.98,"5,900"
7,11/16/2021,₱7.90,₱8.00,₱7.90,₱7.90,"10,300"
8,11/15/2021,₱7.80,₱7.99,₱7.80,₱7.80,"11,200"
9,11/12/2021,₱7.90,₱8.00,₱7.77,₱7.80,"470,200"
